# Imports

In [58]:
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K
from PIL import Image
import numpy as np

In [41]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [42]:
print(x_train.shape, y_train.shape)

(60000, 28, 28) (60000,)


# Preprocessing data

In [43]:
#CNN requires 3-dimension input so we have to reshape the matrix to shape
#(60000, 28, 28, 1)
x_train = x_train.reshape(x_train.shape[0], 28, 28, 1)
x_test = x_test.reshape(x_test.shape[0], 28, 28, 1)
INPUT_SHAPE = (28, 28, 1)

In [44]:
#convert class vectors to categorical data matrix (binary class matrices)
NR_CLASSES = 10 #from 0 to 9
y_train = keras.utils.to_categorical(y_train, NR_CLASSES)
y_test = keras.utils.to_categorical(y_test, NR_CLASSES)

In [45]:
#for easy computations during the process
#we should convert the datatype from uint8 to float32
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
#Rescaling
x_train /= 255.0
x_test /= 255.0

In [46]:
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)

x_train shape: (60000, 28, 28, 1)
x_test shape: (10000, 28, 28, 1)


# Create model

In [47]:
#A CNN model generally consists of convolutional and pooling layers
#It works better for data that are represented as grid structures
#We add dropout layers to deactivate some neurons and reduce overfitting
BATCH_SIZE = 128
NR_EPOCHS = 10
#Initialize our neural networks
model = Sequential()
model.add(Conv2D(filters=32, kernel_size=(3,3), activation='relu',
                input_shape=INPUT_SHAPE))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(NR_CLASSES, activation='softmax'))


In [48]:
#Loss function
loss_function = keras.losses.categorical_crossentropy
#Optimizer
optimization = keras.optimizers.Adadelta()
#compile model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


# Train the model

In [49]:
hist = model.fit(x_train, y_train, batch_size=BATCH_SIZE, epochs=NR_EPOCHS,
                validation_data=(x_test, y_test), verbose=1)
print("Model successfully trained")

Epoch 1/10
469/469 [==============================] - 20s 44ms/step - loss: 0.4180 - accuracy: 0.8672 - val_loss: 0.0649 - val_accuracy: 0.9782
Epoch 2/10
469/469 [==============================] - 23s 48ms/step - loss: 0.1220 - accuracy: 0.9675 - val_loss: 0.0485 - val_accuracy: 0.9855
Epoch 3/10
469/469 [==============================] - 23s 48ms/step - loss: 0.0857 - accuracy: 0.9779 - val_loss: 0.0423 - val_accuracy: 0.9870
Epoch 4/10
469/469 [==============================] - 22s 48ms/step - loss: 0.0692 - accuracy: 0.9823 - val_loss: 0.0375 - val_accuracy: 0.9886
Epoch 5/10
469/469 [==============================] - 23s 49ms/step - loss: 0.0567 - accuracy: 0.9850 - val_loss: 0.0300 - val_accuracy: 0.9910
Epoch 6/10
469/469 [==============================] - 23s 49ms/step - loss: 0.0488 - accuracy: 0.9868 - val_loss: 0.0307 - val_accuracy: 0.9914
Epoch 7/10
469/469 [==============================] - 23s 48ms/step - loss: 0.0433 - accuracy: 0.9886 - val_loss: 0.0297 - val_accuracy:

# Evaluate model

In [54]:
score = model.evaluate(x_test, y_test, verbose=0)
print(f'Test loss: {score[0]:.2f}')
print(f'Test accuracy: {score[1]:.2%}')

Test loss: 0.04
Test accuracy: 99.08%


# Test with test_image

In [87]:
img = Image.open('test_img.png')
img

In [88]:
#Convert image to grayscale
bw = img.convert('L')
#Convert to array
img_array = np.invert(bw)
img_array.shape

(28, 28)

In [91]:
test_img = img_array.reshape((1,28,28,1))
test_img.shape

(1, 28, 28, 1)

In [92]:
res = model.predict([test_img])

In [93]:
print('The predicted digit is:', np.argmax(res[0]))
print(f'With {max(res[0]):.2%} confidence')

The predicted digit is: 2
With 100.00% confidence


# Save model

In [95]:
#model.save('HandwrittenDigitRecognizer.h5')
#model.save('HandwrittenDigitRecognizer.pb')

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: HandwrittenDigitRecognizer.pb\assets
